<a href="https://colab.research.google.com/github/DIFACQUIM/Cursos/blob/main/04_Adquisici%C3%B3n_de_datos_Zinc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***ZINC***

---
---

**Última actualización:** Septiembre 2023

---
---



# **Paqueterias**: instalación e importación

In [2]:
from IPython.utils import io
import tqdm.notebook
import os, sys, random, subprocess
total = 100
with tqdm.notebook.tqdm(total=total) as pbar:
    with io.capture_output() as captured:
        from platform import python_version
        pbar.update(20)
        #Librerias gráficas
        !pip install matplotlib
        import matplotlib.pyplot as plt
        import matplotlib.font_manager as font_manager
        %matplotlib inline
        !pip install seaborn
        import seaborn as sns
        pbar.update(30)
        #Librerias del sistema y herramientas primarias:
        import os.path
        os.getcwd()
        !pip install pandas
        import pandas as pd
        #Conectar con ZINC20
        !pip install molbloom
        import molbloom
        from molbloom import buy
        from molbloom import BloomFilter
        pbar.update(30)
        from tqdm.auto import tqdm
        pbar.update(10)
        # Mount Google Drive and upload your PyMOL license
        pbar.update(10)

  0%|          | 0/100 [00:00<?, ?it/s]

In [3]:
molbloom.catalogs()
#A continuación verás los catalogos disponibles

{'zinc20': 'All ZINC20 (1,006,651,037 mols) from Oct 2021. FPR of 0.003. Requires download',
 'zinc-instock': 'ZINC20 instock (9,227,726 mols). FPR of 0.0003. Requires download',
 'zinc-instock-mini': 'ZINC20 instock (9,227,726 mols). FPR of 0.07. Included in package',
 'surechembl': 'SureChEMBL (22,843,364 mols). FPR of 0.000025. Requires download'}

##**Pruebas**

In [4]:
#buy('CCCO')
# True
buy('CCCO')
# False

Starting zinc-instock download to cache directory /root/.cache/molbloom


True

##**Ejercicio**.
Importar el archivo: "SMILES_prueba.xlsx
Este archivo tiene un listado de SMILES donde únicamente el primero de ellos está disponible en zinc.

In [5]:
id=pd.read_csv('https://raw.githubusercontent.com/DIFACQUIM/Cursos/main/Datasets/Tabla%20para%20disponibilidad%20ZINC.csv')#Lectura del archivo CSV
print(f"El dataframe tiene las siguientes filas y columnas: {id.shape}")
id[0:10]

El dataframe tiene las siguientes filas y columnas: (605, 1)


,SMILES
0,COc1ccc(-c2oc3cc(O)c(OC)c(O)c3c(=O)c2OC)cc1
1,C=C1C(=O)O[C@H]2[C@H]1CC[C@]1(C)[C@@H]2C(C)=CC...
2,COC1=CC(=O)[C@@H]2O[C@]2(C)[C@H]1O
3,COC1=CC(=O)C(O)=C(C)C1=O
4,COC1=C(O)C(=O)C(O)=C(C)C1=O
5,COC1=CC(=O)[C@H](Cl)[C@@](C)(O)[C@H]1O
6,CNC1=CC(=O)C(O)=C(C)C1=O
7,COC1=CC(=O)[C@H](Nc2ccccc2)[C@@](C)(O)[C@H]1O
8,CCCCc1ccc(NC2=C(C)C(=O)C(OC)=CC2=O)cc1
9,COc1cc(-c2cc(=O)c3c(O)c(O)c(OC)cc3o2)ccc1Oc1cc...


In [6]:
def get_availability(table, column, catalog):
    """
    Definición que nos permitirá extraer la información de diferentes catalogos existentes de ZINC cargados por el proyecto: https://pypi.org/project/molbloom/2.0.0/
    """
    num = 0
    df=pd.DataFrame()
    for i in table[column]:
        result = buy(i, catalog)
        df.loc[i, 'vendors'] = result
        if result:
            num += 1
    print(f"Available compounds: {num}, from: {catalog}")
    df['catalog'] = catalog
    df=df.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='').rename(columns={'index':'SMILES'})
    df=df[df['vendors']==True] #Silenciar en caso de querer observar resultados sin vendedores (False)-->silenciar= #df=
    return df

In [7]:
a=get_availability(id, column='SMILES', catalog='zinc20')
b=get_availability(id, column='SMILES', catalog='zinc-instock')
c=get_availability(id, column='SMILES', catalog='zinc-instock-mini')
d=get_availability(id, column='SMILES', catalog='surechembl')

Starting zinc20 download to cache directory /root/.cache/molbloom
Available compounds: 5, from: zinc20
Available compounds: 154, from: zinc-instock
Available compounds: 183, from: zinc-instock-mini
Starting surechembl download to cache directory /root/.cache/molbloom
Available compounds: 142, from: surechembl


In [8]:
df1=pd.merge(a, b, on='SMILES', how='outer').rename(columns={'vendors_x':'Vendor_A','catalog_x': 'Catalog_A', 'vendors_y':'Vendor_B','catalog_y': 'Catalog_B' })
df2=pd.merge(c,d, on='SMILES', how='outer').rename(columns={'vendors_x':'Vendor_C','catalog_x': 'Catalog_C', 'vendors_y':'Vendor_D','catalog_y': 'Catalog_D' })
global_df=pd.merge(df1, df2, on='SMILES', how='outer')
print(global_df.shape)
global_df

(217, 9)


,SMILES,Vendor_A,Catalog_A,Vendor_B,Catalog_B,Vendor_C,Catalog_C,Vendor_D,Catalog_D
0,COc1ccc2ccc(=O)oc2c1,True,zinc20,True,zinc-instock,True,zinc-instock-mini,True,surechembl
1,COc1cc2ccc(=O)oc2cc1OC,True,zinc20,True,zinc-instock,True,zinc-instock-mini,True,surechembl
2,COc1cc2ccc(=O)oc2cc1O,True,zinc20,True,zinc-instock,True,zinc-instock-mini,True,surechembl
3,CC(C)c1cc2cc3ccc(=O)oc3cc2o1,True,zinc20,True,zinc-instock,True,zinc-instock-mini,True,surechembl
4,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,True,zinc20,True,zinc-instock,True,zinc-instock-mini,NaN,NaN
...,...,...,...,...,...,...,...,...,...
212,CC1=CC[C@H]2C(C)(C)CCC[C@]2(C)[C@H]1CC[C@H](C)...,NaN,NaN,NaN,NaN,NaN,NaN,True,surechembl
213,CCCCCc1cc(O)c2c(c1)OC(C)(C)[C@@H]1CC[C@](C)(O)...,NaN,NaN,NaN,NaN,NaN,NaN,True,surechembl
214,COc1cc(O)c2c(c1)OC(c1ccc(OC)c(O)c1)CC2=O,NaN,NaN,NaN,NaN,NaN,NaN,True,surechembl
215,CC(C)=CCc1c2c(c(O)c3c1O[C@H](c1ccc(O)cc1)[C@@H...,NaN,NaN,NaN,NaN,NaN,NaN,True,surechembl


##**Referencia**



* https://pypi.org/project/molbloom/2.0.0/
